In [1]:
import pandas as pd
from datetime import datetime as dt

import integrate as intr


In [2]:

miv_df = intr.process_miv_data()
fb_data = intr.process_foot_bike_data()

/Users/seb/Projects/repos/group-1/src/integrate.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miv_df_cols_dropped['Weekday_en'] = days.map(lambda x: weekday_names[x])


In [3]:
test_df = miv_df


In [4]:
test_df.dtypes
date_object = pd.to_datetime(test_df['Date'])


In [5]:
test_df['Weekday_Name']

KeyError: 'Weekday_Name'

In [6]:
acc_df = intr.process_accident_data()

u_string RoadTrafficAccidentLocations.json
Filename: 2017_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: RoadTrafficAccidentLocations.json
Filepath: datasets/RoadTrafficAccidentLocations.json
Filename: 2016_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: 2022_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: 2015_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: 2019_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: sid_dav_verkehrszaehlung_miv_OD2031_2013.csv
Filename: 2021_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: sid_dav_verkehrszaehlung_miv_OD2031_2012.csv
Filename: sid_dav_verkehrszaehlung_miv_OD2031_2016.csv
Filename: 2014_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: 2018_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: sid_dav_verkehrszaehlung_miv_OD2031_2017.csv
Filename: sid_dav_verkehrszaehlung_miv_OD2031_2015.csv
Filename: 2020_verkehrszaehlungen_werte_fussgaenger_velo.csv
Filename: sid_dav_verkehrszaehlung_m

In [ ]:
acc_df.head()
acc_df['AccidentWeekDay'].unique()
#acc_df.dtypes
date_obj = dt.strptime(acc_df[''])


In [ ]:
print("Accident Columns:")
print(acc_df.dtypes)
print()
print("MIV Columns:")
print(miv_df.dtypes)
print()
print("FB Cols:")
print(fb_data.dtypes)

In [ ]:
print("MIV unqiue:", miv_df['EKoord'].unique().shape)
print("Acc unique:", acc_df['AccidentLocation_CHLV95_E'].unique().shape)
print("FB unique: ", fb_data['OST'].unique())


In [9]:
acc_df.head()

,AccidentUID,AccidentHour,AccidentYear,AccidentWeekDay_en,AccidentType,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,RoadType_en,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,geometry
0,A2D2677533867004E0430A865E337004,00,2011,Saturday,at0,as4,false,false,false,rt433,Minor road,2684605,1245194,POINT(8.55841025 47.3521677)
1,9FD6441F802C20A6E0430A865E3320A6,01,2011,Saturday,at0,as3,false,true,false,rt433,Minor road,2682382,1246980,POINT(8.52932024 47.36851152)
2,9FDA0DC4856A6094E0430A865E336094,02,2011,Saturday,at0,as4,false,false,false,rt439,Other,2682791,1247749,POINT(8.5348767 47.37537618)
3,A3B66E42396E6000E0430A865E336000,02,2011,Saturday,at5,as3,false,false,false,rt433,Minor road,2681199,1247102,POINT(8.51368203 47.36975554)
4,9FDA0DBE8CCE9096E0430A865E339096,03,2011,Saturday,at0,as4,false,false,false,rt433,Minor road,2682479,1250690,POINT(8.53128819 47.40186473)


In [21]:
from sqlalchemy import create_engine
from geoalchemy2 import Geometry, WKTElement
import geopandas as gpd
from shapely import wkt

db_url = f'postgresql://seb:@localhost:5432/test-db23'
engine =  create_engine(db_url)

#miv_df.to_sql('table_name', engine, if_exists='replace', index=False)
#fb_data.to_sql('footbike', engine, if_exists='replace', index=False)

geometry_column = 'geometry'


acc_df['geometry'] = acc_df['geometry'].apply(lambda geom: WKTElement(geom.wkt, srid=4326))

acc_df.to_sql('accidents', engine, if_exists='replace', index=False, dtype={'geometry': Geometry('POINT', srid=4326)})



/var/folders/s3/8bc7ys2d24lgqhdlpttvp70r0000gn/T/ipykernel_59953/958527375.py:15: UserWarning: Geometry column does not contain geometry.
  acc_df['geometry'] = acc_df['geometry'].apply(lambda geom: WKTElement(geom.wkt, srid=4326))


826

In [22]:
engine.dispose()